### Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. 
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. 
Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [1]:
import pandas as pd
data = pd.read_csv('ratings.csv')
data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [2]:
# с помощью value_counts нахожу для каждого пользователя количество оценок, которые он поставил, 
# чтобы в дальнейшем выяснить, кто из них поставил более 100 оценок:
active_users = data['userId'].value_counts()
#получился объект Series, перевожу его в датафрейм, попутно используя reset_index:
df_active_users = pd.DataFrame(active_users).reset_index(level=None, drop=False, inplace=False)
# даю правильные названия столбцам:
df_active_users = df_active_users.set_axis(['userId', 'number_rating'], axis=1, inplace=False)
df_active_users.tail()
#выясняю, что у нас всего 671 ункальный пользователь(чтобы потом никого не потерять)

,userId,number_rating
666,221,20
667,444,20
668,484,20
669,35,20
670,485,20


In [3]:
#аггрегирую min и max, чтобы, используя разницу минимального и максимального занчения timestamp, вычислить время жизни пользователей для каждой оценки
data_grouped = data.groupby('userId').agg([min, max])
data_grouped.head()

movieId        rating        timestamp            
           min    max    min  max         min         max
userId                                                   
1           31   3671    1.0  4.0  1260759108  1260759205
2           10    720    1.0  5.0   835355395   835356246
3           60  84236    2.0  5.0  1298861589  1298932787
4           10   4006    1.0  5.0   949778714   949982274
5            3  48385    1.5  5.0  1163373044  1163375145

In [4]:
# нахожу lifetime
data_grouped['lifetime'] = data_grouped['timestamp']['max'] - data_grouped['timestamp']['min']
data_grouped.head()
#вывожу значение userId из индекса в столбец
data_grouped.reset_index().head()

userId movieId        rating        timestamp             lifetime
             min    max    min  max         min         max         
0      1      31   3671    1.0  4.0  1260759108  1260759205       97
1      2      10    720    1.0  5.0   835355395   835356246      851
2      3      60  84236    2.0  5.0  1298861589  1298932787    71198
3      4      10   4006    1.0  5.0   949778714   949982274   203560
4      5       3  48385    1.5  5.0  1163373044  1163375145     2101

In [5]:
#оставляю только нужные столбцы
df_user_lifetime = data_grouped.reset_index()[['userId', 'lifetime']]
df_user_lifetime.head()

,userId,lifetime
,,
0,1,97
1,2,851
2,3,71198
3,4,203560
4,5,2101


In [21]:
total_result = df_user_lifetime.merge(df_active_users, how='left', on = 'userId')
total_result.tail(10)
# с этим еще до конца не разобралась, почему задваиватется столбец userId, а не сливается и почему названия уходят в скобки c запятой

C:\Users\user\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,userId,"(userId, )","(lifetime, )",number_rating
661,662,662,622,58
662,663,663,15086,26
663,664,664,98180439,519
664,665,665,54131471,434
665,666,666,642,40
666,667,667,1014,68
667,668,668,282,20
668,669,669,685,37
669,670,670,2162705,31
670,671,671,11283984,115


In [18]:
# отсеиваю пользователей,оставляя только тех, кто поставил более 100 оценок
filtered_result = total_result[total_result.number_rating > 100]
filtered_result.head()

,userId,"(userId, )","(lifetime, )",number_rating
3,4,4,203560,204
7,8,8,85187,116
14,15,15,471393496,1700
16,17,17,8053,363
18,19,19,5282,423


In [19]:
#переименовываю корявые названия
df_result = filtered_result.set_axis(['user','dupl_userId', 'active_user_lifetime','rating'], axis=1, inplace=False)
df_result.head()

,user,dupl_userId,active_user_lifetime,rating
3,4,4,203560,204
7,8,8,85187,116
14,15,15,471393496,1700
16,17,17,8053,363
18,19,19,5282,423


In [20]:
#нахожу среднее время жизни пользователей, которые выставили более 100 оценок 
result = df_result['active_user_lifetime'].mean()/24/360
print('Среднее время жизни пользователей, которые выставили более 100 оценок (в днях):', result)

Среднее время жизни пользователей, которые выставили более 100 оценок (в днях): 4638.9476214829165
